In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "/mnt/taurus/data/xixu/llm/llama-2-7b/hf",
    padding_side="right",
    use_fast=False,
)
print(len(tokenizer))

/mnt/taurus/home/xixu/anaconda3/envs/sllama/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


32000


In [1]:
%load_ext autoreload

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
sys.path.append('/mnt/taurus/home/siqiouyang/work/projects/SimulEval/')

import argparse, time, json
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import torch, transformers
import torch.nn as nn
from eval.utils import disable_torch_init
from model.model import SpeechLlamaForCausalLM, SpeechLlamaModel, SpeechLlamaConfig
from model.utils import KeywordsStoppingCriteria
from fairseq.data.audio.speech_to_text_dataset import _collate_frames
from train.dataset import PromptSpeechToTextDatasetCreator, SpeechToTextDatasetItem
import conversation as conversation_lib
from conversation import SeparatorStyle

import requests

import torch.nn.functional as F

import importlib
import numpy as np
from eval.agents.tt_waitk_sllama import S2TAgentStates
from eval.agents.tt_waitk_sllama_incremental import IncrementalS2TAgentStates
from fairseq.models.speech_to_text import lengths_to_padding_mask
from train.uni_wav2vec_monkey_patch import replace_uni_train, replace_uni_decode, uni_self_attn_forward, uni_w2v2_extract_features

transformers.set_seed(998244353)
torch.use_deterministic_algorithms(True)

args = argparse.Namespace()
args.model_name = '/mnt/taurus/data/xixu/runs/sllama/en-es/7b/uni/stage2/checkpoint-2000'
args.length_adapter_path = os.path.join(args.model_name, 'length_adapter.bin')
args.mlp_adapter_path = os.path.join(args.model_name, 'mlp_adapter.bin')
args.speech_tower_path = os.path.join(args.model_name, 'speech_tower.bin')

load_type = torch.float32
disable_torch_init()
model_name = os.path.expanduser(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    use_fast=False,
)
config = json.load(open(os.path.join(args.model_name, 'config.json')))
config['large_model'] = True
update_config = os.path.join(args.model_name, 'config_large.json')
json.dump(config, open(update_config, 'w'), indent=2)
# replace_llama_attn_with_flash_attn()

replace_uni_train()

model = SpeechLlamaForCausalLM.from_pretrained(args.model_name,
                                                torch_dtype=load_type,
                                                low_cpu_mem_usage=True,
                                                device_map='cpu',
                                                config=update_config,).eval()

device_input = device_output = 'cpu'

length_after_ssl, length_after_adp = model.model.initialize_speech_modules(
    speech_tower_path='/mnt/taurus/data/xixu/models/wav2_vec_vox_960h_pl.pt',
    speech_tower_type=None,
    len_adapter_channels=model.config.len_adapter_channels,
    len_adapter_kernel_sizes=model.config.len_adapter_kernel_sizes,
    ssl_fintuned=model.config.ssl_fintuned,
)
model.model.speech_tower.to(dtype=load_type, device=device_input)

length_adapter_weights = torch.load(args.length_adapter_path, map_location='cpu')
mlp_adapter_weights = torch.load(args.mlp_adapter_path, map_location='cpu')
speech_tower_weights = torch.load(args.speech_tower_path, map_location='cpu')


model.model.mm_length_adapter.load_state_dict(length_adapter_weights)
model.model.mm_mlp_adapter.load_state_dict(mlp_adapter_weights)
model.model.speech_tower.load_state_dict(speech_tower_weights)

model.model.mm_length_adapter.to(dtype=load_type, device=device_input).eval()
model.model.mm_mlp_adapter.to(dtype=load_type, device=device_input).eval()
model.model.speech_tower.to(dtype=load_type, device=device_input).eval()

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"
DEFAULT_SPEECH_TOKEN = "<speech>"
DEFAULT_SPEECH_PATCH_TOKEN = "<sp_patch>"
DEFAULT_SPEECH_START_TOKEN = "<sp_start>"
DEFAULT_SPEECH_END_TOKEN = "<sp_end>"

/home/xixu/anaconda3/envs/sllama/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-13 23:48:37,825] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]
Some weights of the model checkpoint at /mnt/taurus/data/xixu/runs/sllama/en-es/7b/uni/stage2/checkpoint-2000 were not used when initializing SpeechLlamaForCausalLM: ['model.speech_tower.encoder.layers.4.final_layer_norm.weight', 'model.speech_tower.encoder.layers.4.self_attn.k_proj.weight', 'model.speech_tower.feature_extractor.conv_layers.0.0.weight', 'model.speech_tower.encoder.layers.10.self_attn.k_proj.bias', 'model.speech_tower.encoder.layers.22.fc2.bias', 'model.speech_tower.encoder.layers.1.fc1.weight', 'model.speech_tower.feature_extractor.conv_layers.2.2.1.bias', 'model.speech_tower.encoder.layers.5.self_attn.k_proj.weight', 'model.speech_tower.encoder.layers.23.self_attn.k_proj.bias', 'model.speech_tower.encoder.layers.2.self_attn_layer_norm.weight', 'model.speech_tower.encoder.layers.20

In [3]:
print(model.config.hidden_size)
# print(length_after_ssl)
print(model.model.embed_tokens.embedding_dim)
print(model.model.embed_tokens)

4096
4096
Embedding(32003, 4096)


In [13]:
import fairseq
state = fairseq.checkpoint_utils.load_checkpoint_to_cpu(model.config.speech_tower_path)
speech_dimension = state['cfg']['model']['w2v_args']['model'].encoder_embed_dim

In [14]:
print(speech_dimension)

1024


In [16]:
test_dataset = PromptSpeechToTextDatasetCreator.from_tsv("/mnt/taurus/data/xixu/datasets/must-c-v1.0/en-es/", 'tst-COMMON_1' )
for test_data in test_dataset:
    source, ref, id = test_data.source, test_data.target, test_data.id                  
    speech_batch = _collate_frames([source], is_audio_input=True)
    n_frames = torch.tensor([source.size(0)], dtype=torch.long)
    speech_lens = length_after_adp(length_after_ssl(n_frames)) # after forward ssl model and length adapter
    print(speech_lens)
    print(n_frames)
    model.eval()
    # Forward pass through the model
    with torch.no_grad():
        result = model.model.get_ssl_feature_w2v(speech_batch, n_frames, speech_lens)
        print(result.shape)

tensor([115])
tensor([149440])
torch.Size([115, 1, 4096])


In [17]:
ctc_head = nn.Linear(4096, 32001) 
result = result.transpose(0, 1).float()
print(result.shape)
result_after_ctc_head = ctc_head(result)
print(result_after_ctc_head.shape)
log_probs = F.log_softmax(result_after_ctc_head, dim=2).transpose(0, 1)
print(log_probs.shape)


torch.Size([1, 115, 4096])
torch.Size([1, 115, 32001])
torch.Size([115, 1, 32001])
